# Class-Based Biological Similarities

Calculate class-based similarities of the Cell Painting Assay (CPA) profiles.  
The profiles have a length of 579 features (all columns starting with `Median_`) and are compared by Distance Correlation (`scipy.spatial.distance`).  
The Similarity is then (1 - Distance Correlation). Negative values are set to zero.  
The Similarity is expressed in percent.  
In addition, the Cell Painting data contains an Induction parameter, which describes the percentage of significantly changed features in the profile (i.e. the percentage of features with abs. values &ge; 3.0).  
This parameter is used to estimate the extent to which the morphological profile is changed compared to the controls.

More details can be found in the SI of the paper.

In [1]:
%reload_ext autoreload
%autoreload 2
def warn(*args, **kwargs):
    pass  # to silence scikit-learn warnings

import warnings
warnings.filterwarnings('ignore')
warnings.warn = warn

# Stdlib Imports
from pathlib import Path
import sys
import functools

# Global package Imports
import pandas as pd
import numpy as np
from scipy.stats import median_absolute_deviation as mad

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Project-local Imports
PROJECT_DIR = list(Path("..").absolute().parents)[1]
sys.path.append(str(PROJECT_DIR))
import plt_style
import cpa
import utils as u
from utils import lp

In [3]:
def class_sim(df, list1, list2=None):  # lists of Well_Ids
    result = []
    if list2 is None: # comparison within one class
        last_index = len(list1) - 1
    else:
        last_index = len(list1)
    for idx1, w_id1 in enumerate(list1[:last_index]):
        if list2 is None:
            for w_id2 in list1[idx1+1:]:
                if w_id1 == w_id2: continue
                result.append(cpa.well_id_similarity(df, w_id1, w_id2))
        else:
            for w_id2 in list2:
                if w_id1 == w_id2: continue
                result.append(cpa.well_id_similarity(df, w_id1, w_id2))
    return result

In [4]:
input_dir = "../Input Data"
!ls "$input_dir"

bio_class_comp_well_ids.tsv
cell_painting_data.tsv
chembl_26_natprot_ids_only.tsv
chembl_np_std_np_scout_values_only.tsv
drugbank_sample_100.tsv
drugbank_std_np_scout_values_only.tsv
drugbank_std_subset_np_scout_values_only.tsv
internal_cpds_mc.tsv
internal_cpds_old_cpd_class.tsv
internal_cpds_std_np_scout.tsv
internal_cpds_std_np_scout_values_only.tsv
internal_cpds_std.tsv
internal_cpds.tsv
README.md


## Load Well_Ids

In [5]:
well_ids = pd.read_csv(f"{input_dir}/bio_class_comp_well_ids.tsv", sep="\t")
lp(well_ids, "Well Ids")

Shape Well Ids                                    :        196 /    3  [ Well_Id, CpdClass, Group ]  


## Merge Cell Painting Data

In [6]:
cp_data = pd.read_csv(f"{input_dir}/cell_painting_data.tsv", sep="\t")

In [7]:
df_org = pd.merge(well_ids, cp_data, on="Well_Id", how="inner")
lp(df_org, "Merged CP data")

Shape Merged CP data                              :        196 /  595  


## Calculate Biological Class Similarities by Group

In [8]:
groups = sorted(df_org["Group"].unique())
print(groups)

['QD', 'QN', 'Sinomenine', 'Sub-classes']


In [ ]:
for group in groups:
    print(group, ":")
    df_tmp = df_org[df_org["Group"] == group].copy()
    cpd_classes = sorted(df_tmp["CpdClass"].unique())
    /lp "df_tmp"
    series_cpd_class1 = []
    series_cpd_class2 = []
    series_sim = []
    for idx1, cpd_class1 in enumerate(cpd_classes):
        for cpd_class2 in cpd_classes[idx1:]:
            list1 = list(df_tmp[df_tmp["CpdClass"] == cpd_class1]["Well_Id"].values)
            list2 = list(df_tmp[df_tmp["CpdClass"] == cpd_class2]["Well_Id"].values)
            sims = class_sim(df_tmp, list1, list2)
            series_cpd_class1.extend([cpd_class1] * len(sims))
            series_cpd_class2.extend([cpd_class2] * len(sims))
            series_sim.extend(sims)

    df_sim_inter = pd.DataFrame({"CpdClass1": series_cpd_class1, "CpdClass2": series_cpd_class2, "BioSim": series_sim})
    df_sim_inter.to_csv(f"results/biosim_inter_{group}.tsv", sep="\t", index=False)
    
    df_sim_inter_grp = df_sim_inter.groupby(by=["CpdClass1", "CpdClass2"]).agg([np.median, mad]).reset_index()
    df_sim_inter_grp.columns = df_sim_inter_grp.columns.map('_'.join)
    df_sim_inter_grp = df_sim_inter_grp.rename(columns={"CpdClass1_": "CpdClass1", "CpdClass2_": "CpdClass2"})
    df_sim_inter_grp.to_csv(f"results/biosim_inter_grp_{group}.tsv", sep="\t", index=False)

    tmp = df_sim_inter_grp.copy()
    tmp["BioSim_median"] = tmp["BioSim_median"] * 100
    tmp = tmp.pivot("CpdClass1", "CpdClass2", "BioSim_median")
    x_size = max(3, 2 + len(cpd_classes))
    y_size = max(1.5, 0.5 + len(cpd_classes) * 0.7)
    print(f"Size: {x_size} / {y_size}")
    fs = (x_size, y_size)
    # plt.rcParams['axes.titlesize'] = 25
    f, ax = plt.subplots(figsize=fs)
    hm = sns.heatmap(tmp, annot=True, fmt=".0f", linewidths=.5, annot_kws={"size": 16}, cmap="YlGnBu", vmin=0.0, vmax=100.0, ax=ax)
    hm.invert_yaxis()
    hm.set_xticklabels(hm.get_xticklabels(), rotation=45, horizontalalignment='right')
    hm.set_yticklabels(hm.get_yticklabels(), rotation=0)
    hm.set_title("Inter-Class Biol. Sim. [%]", pad=20)
    fig = hm.get_figure()
    fig.savefig(f"plots/bio_sim_inter_{group}.png", bbox_inches='tight');